## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()
# show how many unique values there are for the Department column
y_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'NumCompaniesWorked', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'PercentSalaryHike', 'YearsSinceLastPromotion', 'StockOptionLevel', 'TrainingTimesLastYear']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
NumCompaniesWorked          int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
PercentSalaryHike           int64
YearsSinceLastPromotion     int64
StockOptionLevel            int64
TrainingTimesLastYear       int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [8]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

# Fit the encoder to the training data
enc.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
encoded_y_train = enc.transform(y_train['Department'].values.reshape(-1, 1))
encoded_y_test = enc.transform(y_test['Department'].values.reshape(-1, 1))



In [9]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder()

# Fit the encoder to the training data
enc.fit(y_train['Attrition'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
encoded_y_train_attrition = enc.transform(y_train['Attrition'].values.reshape(-1, 1))
encoded_y_test_attrition = enc.transform(y_test['Attrition'].values.reshape(-1, 1))


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_features = X_train_scaled.shape[1]
# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=32, activation='relu')(shared_layer1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer
department_branch = shared_layer2

# Create the hidden layer
hidden_layer_department = layers.Dense(units=3, activation='relu')(department_branch)

# Create the output layer
output_layer_department = layers.Dense(units=3, activation='softmax', name='department_output')(hidden_layer_department)

In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_branch = shared_layer2
# Create the hidden layer
hidden_layer_attrition = layers.Dense(units=2, activation='relu')(attrition_branch)

# Create the output layer
output_layer_attrition = layers.Dense(units=1, activation='sigmoid', name='attrition_output')(hidden_layer_attrition)

In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[output_layer_department, output_layer_attrition])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 12)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   832       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 3)                    99        ['dense_1[0][0]']             
                                                                                             

In [14]:
# Define the variables y_train_department and y_train_attrition and convert y_train_attrition to 0s and 1s
y_train_department = encoded_y_train.toarray()
y_train_attrition = encoded_y_train_attrition.toarray()

# Modify the shape of y_train_attrition to match the output layer shape
y_train_attrition = encoded_y_train_attrition.toarray()[:, 1]
y_test_attrition = encoded_y_test_attrition.toarray()[:, 1]

# Train the model
model.fit(X_train_scaled, [y_train_department, y_train_attrition], epochs=100, verbose=2)


Epoch 1/100


35/35 - 1s - loss: 1.8020 - department_output_loss: 1.0657 - attrition_output_loss: 0.7363 - department_output_accuracy: 0.4936 - attrition_output_accuracy: 0.5590 - 901ms/epoch - 26ms/step
Epoch 2/100
35/35 - 0s - loss: 1.5853 - department_output_loss: 0.9069 - attrition_output_loss: 0.6784 - department_output_accuracy: 0.6570 - attrition_output_accuracy: 0.8212 - 29ms/epoch - 829us/step
Epoch 3/100
35/35 - 0s - loss: 1.4649 - department_output_loss: 0.7973 - attrition_output_loss: 0.6676 - department_output_accuracy: 0.6661 - attrition_output_accuracy: 0.8267 - 29ms/epoch - 829us/step
Epoch 4/100
35/35 - 0s - loss: 1.4173 - department_output_loss: 0.7596 - attrition_output_loss: 0.6578 - department_output_accuracy: 0.6652 - attrition_output_accuracy: 0.8276 - 29ms/epoch - 829us/step
Epoch 5/100
35/35 - 0s - loss: 1.3963 - department_output_loss: 0.7481 - attrition_output_loss: 0.6482 - department_output_accuracy: 0.6670 - attrition_output_accuracy: 0.8285 - 28ms/epoch -

In [15]:
# Evaluate the model with the testing data
total_loss, loss_department, loss_attrition, accuracy_department, accuracy_attrition = model.evaluate(X_test_scaled, [encoded_y_test.toarray(), y_test_attrition], verbose=2)


12/12 - 0s - loss: 2.0592 - department_output_loss: 1.6672 - attrition_output_loss: 0.3920 - department_output_accuracy: 0.5217 - attrition_output_accuracy: 0.8723 - 168ms/epoch - 14ms/step


In [16]:
# Calculate and print the accuracy for both department and attrition
_, _, _, accuracy_department, accuracy_attrition = model.evaluate(X_test_scaled, [encoded_y_test.toarray(), y_test_attrition], verbose=2)
print(f"Department Accuracy: {accuracy_department}")
print(f"Attrition Accuracy: {accuracy_attrition}")


12/12 - 0s - loss: 2.0592 - department_output_loss: 1.6672 - attrition_output_loss: 0.3920 - department_output_accuracy: 0.5217 - attrition_output_accuracy: 0.8723 - 24ms/epoch - 2ms/step
Department Accuracy: 0.52173912525177
Attrition Accuracy: 0.8722826242446899


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I do not think so. The data consists of 3 different departments (with disproportionate representation), utilizing 10 selected columns to assess attrition. Our department accuracy was quite low compared to attrition accuracy. The disparity in accuracy is most likely due to the fact that the 3 unique values for department are very unbalanced.
2. I used softmax for department because it has 3 variables, being sales, research and development, and human resources. I used sigmoid for attrition because it is a binary output of either yes, or no. 
3. Balancing the data, separating each department into its own dataframe for assessment, prioritizing the 10 selected assessment columns for outcome relevance.